<img src=".\images\Logo.png" width=150 align="left" /> <img src=".\images\Logo2.jpg" width=450 align="right" />


# <center><font color= #1e8449 > <b>CU18_Infraestructuras_eventos</font></center>



<font color='steelblue'><b>Citizenlab Data Science Methodology > II - Data Processing Domain </font>
***
> # <font color='steelblue'> <b>05.- Data Collection</font>

<font color='steelblue'>Data Collection is the process to obtain and generate (if required) necessary data to model the problem.</font>

### 20. Interpolar variables clima a infraestructuras

* A partir de los datos de las estaciones de medición AEMET, se interpolan por kriging los valores estimados en cada infraestructura







<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Settings" data-toc-modified-id="Settings-1"><font color="green">Settings</font></a></span></li><li><span><a href="#Data-Load" data-toc-modified-id="Data-Load-2"><font color="green">Data Load</font></a></span></li><li><span><a href="#ETL-Processes" data-toc-modified-id="ETL-Processes-3">ETL Processes</a></span><ul class="toc-item"><li><span><a href="#Import-data-from:-CSV,-Excel,-Tab,-JSON,-SQL,-and-Parquet-files" data-toc-modified-id="Import-data-from:-CSV,-Excel,-Tab,-JSON,-SQL,-and-Parquet-files-3.1">Import data from: CSV, Excel, Tab, JSON, SQL, and Parquet files</a></span></li></ul></li><li><span><a href="#Synthetic-Data-Generation" data-toc-modified-id="Synthetic-Data-Generation-4">Synthetic Data Generation</a></span></li><li><span><a href="#Fake-Data-Generation" data-toc-modified-id="Fake-Data-Generation-5">Fake Data Generation</a></span></li><li><span><a href="#Open-Data" data-toc-modified-id="Open-Data-6">Open Data</a></span></li><li><span><a href="#Data-Save" data-toc-modified-id="Data-Save-7"><font color="green">Data Save</font></a></span></li><li><span><a href="#Main-Conclusions" data-toc-modified-id="Main-Conclusions-8"><font color=" #2874a6 ">Main Conclusions</font></a></span></li><li><span><a href="#Main-Actions" data-toc-modified-id="Main-Actions-9"><font color=" #2874a6 ">Main Actions</font></a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Acciones-done" data-toc-modified-id="Acciones-done-9.0.1"><font color=" #2874a6 ">Acciones done</font></a></span></li><li><span><a href="#Acctions-to-perform" data-toc-modified-id="Acctions-to-perform-9.0.2"><font color=" #2874a6 ">Acctions to perform</font></a></span></li></ul></li></ul></li></ul></div>

## <font color='green'>Settings</font>

### Packages to use


- {tcltk} para selección interactiva de archivos locales
- {sf} para trabajar con georeferenciación
- {readr} para leer y escribir archivos csv
- {dplyr} para explorar datos
- {tidyr} para organización de datos
- {gstat} para operaciones geoestadísticas


In [1]:
library(readr)
library(dplyr)
library(tidyr)
library(sf)
library(gstat)

p <- c("tcltk", "sf", "readr", "dplyr", "tidyr", "gstat")



Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Linking to GEOS 3.10.2, GDAL 3.4.2, PROJ 8.2.1; sf_use_s2() is TRUE



### Paths

In [2]:
iPath <- "Data/Input/"
oPath <- "Data/Output/"

## <font color='green'>Data Load</font>

If there are more than one input file, make as many sections as files to import.

Instrucciones
- Los ficheros de entrada del proceso están siempre en Data/Input/.   
- Si hay más de un fichero de entrada, se crean tantos objetos iFile_xx y file_data_xx como ficheros de entrada (xx número correlativo con dos dígitos, rellenar con ceros a la izquierda)

1. Datos estaciones AEMET

<font color='tomato'><b> OPCION A:</b> Seleccionar fichero en ventana para mayor comodidad</font> 

Data load using the {tcltk} package. Ucomment the line if not using this option

In [3]:
# file_data_01 <- tcltk::tk_choose.files(multi = FALSE)

<font color='tomato'><b> OPCION B:</b> Especificar el nombre de archivo</font>



In [4]:
iFile_01 <- "CU_18_05_14_aemet.csv"
file_data_01 <- paste0(iPath, iFile_01)

if(file.exists(file_data_01)){
    cat("Se leerán datos del archivo: ", file_data_01)
} else{
    warning("Cuidado: el archivo no existe.")
}


Se leer<U+00E1>n datos del archivo:  Data/Input/CU_18_05_14_aemet.csv

#### Data file to dataframe

Usar la función adecuada según el formato de entrada (xlsx, csv, json, ...)

In [5]:
data_01 <- read_csv(file_data_01)


Rows: 4547 Columns: 22
-- Column specification --------------------------------------------------------
Delimiter: ","
chr   (9): indicativo, nombre, provincia, horatmin, horatmax, dir, horaracha...
dbl  (12): altitud, tmed, prec, tmin, tmax, velmedia, racha, sol, presMax, p...
date  (1): fecha

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


Estructura de  los datos:

In [6]:
glimpse(data_01)

Rows: 4,547
Columns: 22
$ fecha       <date> 2019-01-01, 2019-01-02, 2019-01-03, 2019-01-04, 2019-01-0~
$ indicativo  <chr> "2462", "2462", "2462", "2462", "2462", "2462", "2462", "2~
$ nombre      <chr> "PUERTO DE NAVACERRADA", "PUERTO DE NAVACERRADA", "PUERTO ~
$ provincia   <chr> "MADRID", "MADRID", "MADRID", "MADRID", "MADRID", "MADRID"~
$ altitud     <dbl> 1894, 1894, 1894, 1894, 1894, 1894, 1894, 1894, 1894, 1894~
$ tmed        <dbl> 6.4, 5.2, 7.6, 6.0, 7.2, 4.6, 4.1, 6.2, -0.8, -4.1, -5.2, ~
$ prec        <dbl> 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0~
$ tmin        <dbl> 1.6, -0.8, 2.7, 1.7, 1.8, 0.9, 1.2, 1.4, -3.5, -7.5, -10.3~
$ horatmin    <chr> "23:40", "08:00", "23:59", "01:30", "18:00", "07:10", "Var~
$ tmax        <dbl> 11.3, 11.3, 12.6, 10.2, 12.5, 8.2, 7.0, 10.9, 2.0, -0.7, -~
$ horatmax    <chr> "12:40", "12:40", "12:00", "12:10", "12:50", "11:40", "13:~
$ dir         <chr> "36", "04", "15", "15", "03", "35", "36", "02", "36", "36"~
$ velmedia    <d

Muestra de datos:

In [7]:
slice_head(data_01, n = 5)

fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,...,velmedia,racha,horaracha,sol,presMax,horaPresMax,presMin,horaPresMin,X,Y
<date>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,...,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>
2019-01-01,2462,PUERTO DE NAVACERRADA,MADRID,1894,6.4,0,1.6,23:40,11.3,...,3.3,8.9,23:59,7.9,823.1,10,821.7,05,-4.010556,40.79306
2019-01-02,2462,PUERTO DE NAVACERRADA,MADRID,1894,5.2,0,-0.8,08:00,11.3,...,2.8,11.4,17:10,7.9,822.0,00,818.8,24,-4.010556,40.79306
2019-01-03,2462,PUERTO DE NAVACERRADA,MADRID,1894,7.6,0,2.7,23:59,12.6,...,1.9,8.9,04:00,7.3,819.5,10,818.0,05,-4.010556,40.79306
2019-01-04,2462,PUERTO DE NAVACERRADA,MADRID,1894,6.0,0,1.7,01:30,10.2,...,1.4,8.3,00:40,8.0,822.7,23,818.5,05,-4.010556,40.79306
2019-01-05,2462,PUERTO DE NAVACERRADA,MADRID,1894,7.2,0,1.8,18:00,12.5,...,3.1,8.3,06:50,8.2,824.3,10,822.4,Varias,-4.010556,40.79306


2. Datos de infraestructuras (con identificador)

<font color='tomato'><b> OPCION A:</b> Seleccionar fichero en ventana para mayor comodidad</font> 

Data load using the {tcltk} package. Ucomment the line if not using this option

In [8]:
# file_data_02 <- tcltk::tk_choose.files(multi = FALSE)

<font color='tomato'><b> OPCION B:</b> Especificar el nombre de archivo</font>



In [9]:
iFile_02 <- "CU_18_05_19_01_infraestructuras.csv"
file_data_02 <- paste0(iPath, iFile_02)

if(file.exists(file_data_02)){
    cat("Se leerán datos del archivo: ", file_data_02)
} else{
    warning("Cuidado: el archivo no existe.")
}


Se leer<U+00E1>n datos del archivo:  Data/Input/CU_18_05_19_01_infraestructuras.csv

#### Data file to dataframe

Usar la función adecuada según el formato de entrada (xlsx, csv, json, ...)

In [10]:
data_02 <- read_csv(file_data_02)


Rows: 1138 Columns: 19
-- Column specification --------------------------------------------------------
Delimiter: ","
chr  (8): grupo, tipo, nombre, CODMUN, DIRECCION, info, CMUN, CDIS
dbl (11): id_inf, dist_aeropuerto, dist_intercambiador, dist_cercanias, dist...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


Estructura de  los datos:

In [11]:
glimpse(data_02)

Rows: 1,138
Columns: 19
$ id_inf              <dbl> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,~
$ grupo               <chr> "Transporte", "Transporte", "Transporte", "Transpo~
$ tipo                <chr> "Intercambiadores", "Intercambiadores", "Intercamb~
$ nombre              <chr> "Grandes Intercambiadores Plaza El<U+00ED>ptica", "Grande~
$ CODMUN              <chr> "079", "079", "079", "079", "079", "079", "079", "~
$ DIRECCION           <chr> "Plaza El<U+00ED>ptica s/n", "Calle Princesa, 89", "Estac~
$ info                <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA~
$ CMUN                <chr> "079", "079", "079", "079", "079", "079", "079", "~
$ CDIS                <chr> "12", "09", "09", "05", "05", "05", "06", "02", "0~
$ dist_aeropuerto     <dbl> 5.2031430, 8.5507475, 7.3002463, 9.9339069, 9.1988~
$ dist_intercambiador <dbl> 0.0000000, 0.0000000, 0.0000000, 0.0000000, 0.0000~
$ dist_cercanias      <dbl> 1.648973148, 1.468638183, 0.088338692, 0.736877403~
$ 

Muestra de datos:

In [12]:
slice_head(data_02, n = 5)

id_inf,grupo,tipo,nombre,CODMUN,DIRECCION,info,CMUN,CDIS,dist_aeropuerto,dist_intercambiador,dist_cercanias,dist_hospital,dist_universidad,dist_gob,dist_embajada,dist_ccomercial,X,Y
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Transporte,Intercambiadores,Grandes Intercambiadores Plaza El<U+00ED>ptica,079,Plaza El<U+00ED>ptica s/n,NA,079,12,5.203143,0,1.64897315,1.8154156,1.29960755,0.6177235,0.39213238,0.7311571,-3.716577,40.38540
2,Transporte,Intercambiadores,Grandes Intercambiadores Moncloa,079,"Calle Princesa, 89",NA,079,09,8.550748,0,1.46863818,0.4221115,0.30460781,0.6263847,1.45739694,1.6105674,-3.719508,40.43474
3,Transporte,Intercambiadores,Grandes Intercambiadores Pr<U+00ED>ncipe P<U+00ED>o,079,Estaci<U+00F3>n Pr<U+00ED>ncipe P<U+00ED>o,NA,079,09,7.300246,0,0.08833869,1.3855767,0.92076145,1.1700512,5.62455975,0.9060857,-3.719560,40.42080
4,Transporte,Intercambiadores,Grandes Intercambiadores Plaza de Castilla,079,Calle San Aquilino y Avenida de Asturias,NA,079,05,9.933907,0,0.73687740,1.0733484,0.08835165,0.1490496,0.08316773,0.7758344,-3.689044,40.46719
5,Transporte,Intercambiadores,Grandes Intercambiadores Avenida de Am<U+00E9>rica,079,Avenida de Am<U+00E9>rica 9 A,NA,079,05,9.198897,0,1.62310148,0.3488060,0.50695045,1.1979177,0.40303107,0.9512580,-3.676731,40.43797


3. Datos del diario de infraestructuras

<font color='tomato'><b> OPCION A:</b> Seleccionar fichero en ventana para mayor comodidad</font> 

Data load using the {tcltk} package. Ucomment the line if not using this option

In [ ]:
# file_data_03 <- tcltk::tk_choose.files(multi = FALSE)

<font color='tomato'><b> OPCION B:</b> Especificar el nombre de archivo</font>



In [23]:
iFile_03 <- "CU_18_05_19_02_diario_infra.csv"
file_data_03 <- paste0(iPath, iFile_03)

if(file.exists(file_data_03)){
    cat("Se leerán datos del archivo: ", file_data_03)
} else{
    warning("Cuidado: el archivo no existe.")
}


Se leer<U+00E1>n datos del archivo:  Data/Input/CU_18_05_19_02_diario_infra.csv

#### Data file to dataframe

Usar la función adecuada según el formato de entrada (xlsx, csv, json, ...)

In [24]:
data_03 <- read_csv(file_data_03)


Rows: 415370 Columns: 6
-- Column specification --------------------------------------------------------
Delimiter: ","
dbl  (5): id_inf, capacidad, demanda, evento_infra, evento_zona
date (1): fecha

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


Estructura de  los datos:

In [25]:
glimpse(data_03)

Rows: 415,370
Columns: 6
$ id_inf       <dbl> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17~
$ fecha        <date> 2019-01-01, 2019-01-01, 2019-01-01, 2019-01-01, 2019-01-~
$ capacidad    <dbl> 993, 996, 1036, 1020, 992, 1026, 1007, 976, 1037, 972, 94~
$ demanda      <dbl> 883, 888, 922, 1134, 1103, 1139, 897, 1086, 1150, 861, 83~
$ evento_infra <dbl> 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, ~
$ evento_zona  <dbl> 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, ~


Muestra de datos:

In [26]:
slice_head(data_03, n = 5)

id_inf,fecha,capacidad,demanda,evento_infra,evento_zona
<dbl>,<date>,<dbl>,<dbl>,<dbl>,<dbl>
1,2019-01-01,993,883,1,1
2,2019-01-01,996,888,0,0
3,2019-01-01,1036,922,0,0
4,2019-01-01,1020,1134,1,0
5,2019-01-01,992,1103,1,1


## ETL Processes

### Import data from: CSV, Excel, Tab, JSON, SQL, and Parquet files

Se han importado en el apartado Data Load anterior:

* Datos de infraestructuras tratados con anterioridad
* Datos meteorológicos

Incluir apartados si procede para: Extracción de datos (select, filter), Transformación de datos, (mutate, joins, ...). Si es necesario tratar datos perdidos, indicarlo también en NB 09.2

Si no aplica: Estos datos no requieren tareas de este tipo.

#### Data transformation

* Convertir tablas de datos en objetos espaciales para poder interpolar

In [13]:
tdata_01 <- data_01 |> 
  st_as_sf(coords = c("X", "Y"), crs = 4326) 
tdata_02 <- data_02 |> 
  st_as_sf(coords = c("X", "Y"), crs = 4326) 


* Interpolar temperatura media, precipitación, velocidad media del viento y presión máxima en las coordenadas de cada infraestructura.

NOTA: esta operación puede tardar varios minutos.

In [21]:
meteovars <- c("tmed", "prec", "velmedia", "presMax")
fechas <- tdata_01 |> pull(fecha) |> unique()
resl <- list()

for (i in seq_along(fechas)){
  f <- fechas[i]
#   cat("Calculando", as.character(f), "\n")
  meteod <- tdata_01 |> 
    filter(fecha == f) |> 
    select(all_of(meteovars)) 
  
  res <- tdata_02 |> select(id_inf) |> st_drop_geometry()
  for (j in seq_along(meteovars)){
    thisvar <- meteovars[j] 
    meteodc <- meteod |> 
      drop_na(all_of(thisvar))
    
    if(all(meteodc |> pull(thisvar) == 0)){
      res <- res |> 
        mutate(!!thisvar := 0)
    } else{
      
      fo <- as.formula(paste0(thisvar, " ~ 1"))
      v0 <- variogram(fo, meteodc)
      v.m <- suppressWarnings(fit.variogram(v0, 
        vgm(c("Exp", "Mat", "Sph", "Log", "Wav", "Bes", "Lin", "Leg"))))
      invisible(capture.output(suppressWarnings(krg <- krige(fo, 
                   locations = meteodc,
                   newdata = tdata_02,
                   model = v.m))))
      res <- res |> 
        mutate(!!thisvar := krg$var1.pred)
    }
  }
  res <- res |> 
    mutate(fecha = f)
  resl[[i]] <- res
}

data <- bind_rows(resl)

[1] "a possible solution MIGHT be to scale semivariances and/or distances"
[1] "a possible solution MIGHT be to scale semivariances and/or distances"
[1] "a possible solution MIGHT be to scale semivariances and/or distances"
[1] "a possible solution MIGHT be to scale semivariances and/or distances"
[1] "a possible solution MIGHT be to scale semivariances and/or distances"
[1] "a possible solution MIGHT be to scale semivariances and/or distances"
[1] "a possible solution MIGHT be to scale semivariances and/or distances"
[1] "a possible solution MIGHT be to scale semivariances and/or distances"
[1] "a possible solution MIGHT be to scale semivariances and/or distances"
[1] "a possible solution MIGHT be to scale semivariances and/or distances"
[1] "a possible solution MIGHT be to scale semivariances and/or distances"
[1] "a possible solution MIGHT be to scale semivariances and/or distances"
[1] "a possible solution MIGHT be to scale semivariances and/or distances"
[1] "a possible solution 

In [28]:
glimpse(data)

Rows: 415,370
Columns: 6
$ id_inf   <dbl> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18~
$ tmed     <dbl> 6.953211, 6.196420, 6.483569, 5.875797, 6.212680, 5.878542, 6~
$ prec     <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0~
$ velmedia <dbl> 0.6433886, 0.3417523, 0.4132169, 0.1820178, 0.2118110, 0.1742~
$ presMax  <dbl> 952.9357, 950.2191, 950.8051, 951.6768, 953.5118, 952.1681, 9~
$ fecha    <date> 2019-01-01, 2019-01-01, 2019-01-01, 2019-01-01, 2019-01-01, ~


* Unir datos meteorológicos al diario de infraestructuras

In [29]:
tdata <- data_03 |>
    inner_join(data, by = c("id_inf", "fecha"))

In [30]:
glimpse(tdata)

Rows: 415,370
Columns: 10
$ id_inf       <dbl> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17~
$ fecha        <date> 2019-01-01, 2019-01-01, 2019-01-01, 2019-01-01, 2019-01-~
$ capacidad    <dbl> 993, 996, 1036, 1020, 992, 1026, 1007, 976, 1037, 972, 94~
$ demanda      <dbl> 883, 888, 922, 1134, 1103, 1139, 897, 1086, 1150, 861, 83~
$ evento_infra <dbl> 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, ~
$ evento_zona  <dbl> 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, ~
$ tmed         <dbl> 6.953211, 6.196420, 6.483569, 5.875797, 6.212680, 5.87854~
$ prec         <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ~
$ velmedia     <dbl> 0.6433886, 0.3417523, 0.4132169, 0.1820178, 0.2118110, 0.~
$ presMax      <dbl> 952.9357, 950.2191, 950.8051, 951.6768, 953.5118, 952.168~


In [31]:
tdata |> slice_head(n = 5)

id_inf,fecha,capacidad,demanda,evento_infra,evento_zona,tmed,prec,velmedia,presMax
<dbl>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2019-01-01,993,883,1,1,6.953211,0,0.6433886,952.9357
2,2019-01-01,996,888,0,0,6.196420,0,0.3417523,950.2191
3,2019-01-01,1036,922,0,0,6.483569,0,0.4132169,950.8051
4,2019-01-01,1020,1134,1,0,5.875797,0,0.1820178,951.6768
5,2019-01-01,992,1103,1,1,6.212680,0,0.2118110,953.5118


## Synthetic Data Generation

No aplica

## Fake Data Generation

No aplica

## Open Data

Los datos originales procedían de fuentes abiertas

## <font color='green'>Data Save</font>

Este proceso, puede copiarse y repetirse en aquellas partes del notebbok que necesiten guardar datos.
Recuerde cambiar las cadenas añadida del fichero para diferenciarlas

<font color='tomato'> Identificamos los datos a guardar</font>

In [32]:
data_to_save <- tdata


<font color='tomato'>Estructura de nombre de archivos:</font>

* Código del caso de uso, por ejemplo "CU_04"
* Número del proceso que lo genera, por ejemplo "_05".
* Número de la tarea que lo genera, por ejemplo "_01"
* En caso de generarse varios ficheros en la misma tarea, llevarán _01 _02 ... después
* Nombre: identificativo de "properData", por ejemplo "_zonasgeo"
* Extensión del archivo

Ejemplo: "CU_04_05_01_01_zonasgeo.json, primer fichero que se genera en la tarea 01 del proceso 05 (Data Collection) para el caso de uso 04 (vacunas)

Importante mantener los guiones bajos antes de proceso, tarea, archivo y nombre

### Proceso 05

In [33]:
caso <- "CU_18"
proceso <- '_05'
tarea <- "_20"
archivo <- ""
proper <- "_diario_infra"
extension <- ".csv"

<font color='tomato'><b> OPCION A:</b> Uso del paquete "tcltk" para mayor comodidad</font>

* Buscar carpeta, escribir nombre de archivo SIN extensión (se especifica en el código)
* Especificar sufijo2 si es necesario
* Cambiar datos por datos_xx si es necesario

In [ ]:
# file_save <- paste0(caso, proceso, tarea, tcltk::tkgetSaveFile(), proper, extension) 
# path_out <- paste0(oPath, file_save)
# write_csv(data_to_save, path_out)

# cat('File saved as: ')
# path_out

<font color='tomato'><b> OPCION B:</b> Especificar el nombre de archivo</font>

- Los ficheros de salida del proceso van siempre a Data/Output/.  

In [34]:
file_save <- paste0(caso, proceso, tarea, archivo, proper, extension) 
path_out <- paste0(oPath, file_save)
write_csv(data_to_save, path_out)

cat('File saved as: ')
path_out

File saved as: 

[1] "Data/Output/CU_18_05_20_diario_infra.csv"

#### Copia del fichero a Input

Si el archivo se va a usar en otros notebooks, copiar a la carpeta Input

In [35]:
path_in <- paste0(iPath, file_save)
file.copy(path_out, path_in, overwrite = TRUE)

[1] TRUE

## <font color=' #2874a6 '>Main Conclusions</font>

<font color='steelblue'>List and describe the general conclusions of the analysis carried out​.</font>

### Prerequisites

Para que funcione este código se necesita:

* Las rutas de archivos `Data/Input` y `Data/Output` deben existir (relativas a la ruta del _notebook_)
* El paquete tcltk instalado para seleccionar archivos interactivamente. No se necesita en producción.
* Los paquetes tcltk, sf, readr, dplyr, tidyr, gstat deben estar instalados.


In [36]:
cat("Los paquetes", paste(p, collapse = ", "), "deben estar instalados.")

Los paquetes tcltk, sf, readr, dplyr, tidyr, gstat deben estar instalados.

### Configuration Management

This notebook has been tested with the following versions of R and packages. It cannot be assured that later versions work in the same way:

* R 4.2.2
* tcltk 4.2.2
* sf 1.0.9
* readr 2.1.3
* dplyr 1.0.10
* tidyr 1.3.0
* gstat 2.1.0

### Data structures

#### Objeto `tdata`

- Los datos de origen son las ubicaciones de infraestructuras y los datos de las estaciones meteorológicas
- Hay 415370 filas con las variables:
	* id_inf
	* fecha
	* capacidad
	* demanda
	* evento_infra
	* evento_zona
	* tmed
	* prec
	* velmedia
	* presMax


#### Observaciones generales sobre los datos

* Las estimaciones por interpolación pueden resultar en valores no válidos

### Consideraciones para despliegue en piloto

* Los datos de capacidad y demanda son simulados mientras no se tengan datos completos en las tareas anteriores

### Consideraciones para despliegue en producción

* Se deben crear los procesos ETL en producción necesarios para que los datos de entrada estén actualizados

## <font color=' #2874a6 '>Main Actions</font>

#### <font color=' #2874a6 '>Acciones done</font>

<font color='steelblue'>Indicate the actions that have been carried out in this process</font>

- Se han estimado las variables meteorológicas en las coordenadas de cada infraestructura para cada día de 2019
- Se han unido las estimaciones al diario de infraestructuras con datos simulados

#### <font color=' #2874a6 '>Acctions to perform</font>

<font color='steelblue'>Indicate the actions that must be carried out in subsequent processes</font>

- Se debe comprobar y en su caso corregir la inconsistencia en las estimaciones (por ejemplo, precipitación o velocidad menor de cero)

## <font color='RED'>CODE TO DEPLOY (PILOT)</font>

A continuación se incluirá el código que deba ser llevado a despliegue para producción, dado que se entiende efectúa operaciones necesarias sobre los datos en la ejecución del prototipo

<font color=' red '><b>Description</b></font>

- No hay código que desplegar en el piloto, ya que estos datos son estáticos para el piloto (2019).
- El fichero de salida se debe incluir en la ruta de datos de la implementación ya que será utilizado en los modelos



<font color=' red '><b>CODE</b></font>

In [ ]:
# incluir código